In [6]:
from copy import deepcopy
import numpy as np
import pandas as pd
import json, re, nltk
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("talk")

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
scopus = pd.read_csv("scopus.csv")
keyword_org_name = "keywords.txt"
keyword_abb_name = "keyword_abb.json"
keyword_dash_name = "keywords_dash.txt"
keyword_dict_name = "keywords_dict.json"
keyword_singe_name = "keywords_single.txt"
keyword_plural_name = "keywords_plural.json"

In [4]:
keywords = []
line = ''

with open("BIPV_ML_all.txt") as f:
    while line != "EF":
        line = f.readline()
        if line[:3] == "DE ":
            keywords_new = line[3:].rstrip('\n').split(";")
            for keyword_new in keywords_new:
                keywords += [keyword_new.rstrip(' ').lstrip(' ')]

keywords = sorted(keywords)
with open(keyword_org_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords:
        f.write(f"{keyword}\n")

In [58]:
ord('A')

65

In [67]:
# retreive keywords
keywords_raw_ = np.genfromtxt("keywords.txt", dtype="str", delimiter="\n", encoding='utf-8-sig') #utf-8-sig for \ufeff removal

# get unique ones
keywords_raw_ = np.unique(keywords_raw_)

# abbriviations detection
keywords_abb = {}
keywords_abb_key_cand = []
for keyword_raw_ in keywords_raw_:
    kwr = keyword_raw_.split()
    for kw in kwr:
        if (kw == kw.upper()) and (kw.lstrip("(").rstrip(")") not in keywords_abb_key_cand) and (ord(kw[0]) >= ord('A') and ord(kw[0]) <= ord('Z')):
            keywords_abb_key_cand.append(kw.lstrip("(").rstrip(")"))

# convert to lower cases
keywords_raw = [kw.lower() for kw in keywords_raw_]

# duplicate for values
keywords_value = deepcopy(keywords_raw)

### keywords cleaning
keywords_dash = [] # additional dictionary, for words containing '-'

# remove staring characters
keywords_value = [kw.lstrip('-') for kw in keywords_value]
keywords_value = [kw.rstrip('-') for kw in keywords_value]
keywords_value = [kw.lstrip(':') for kw in keywords_value]
keywords_value = [kw.rstrip(':') for kw in keywords_value]

# remove brakets "()"
for kc in keywords_abb_key_cand:
    print(f"kc= ({kc.lower()})")
    for kw in keywords_value:
        if kw in ["(psa)", "(fpsa)", "(co2)", "(dc)", "(pv)"]: ##### here
            print("kw= ", kw)
        if f"({kc.lower()})" == kw:
            kc_value = re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ')
            if (kc not in list(keywords_abb.keys())) or (kc in list(keywords_abb.keys()) and len(kc_value) < keywords_abb[kc]):
                keywords_abb.update({kc:kc_value})
            
keywords_value = [re.sub(r'\([^)]*\)', "", kw).rstrip(' ').lstrip(' ') for kw in keywords_value]

# remove unicodes –
keywords_value = [kw.replace("‐", "-") for kw in keywords_value] # \u2010 hypen
keywords_value = [kw.replace("–", "-") for kw in keywords_value] # \u2013 dash 
keywords_value = [kw.replace("—", "-") for kw in keywords_value] # \u2014 dash 
keywords_value = [kw.replace("’", "-") for kw in keywords_value] # \u2019 right quotation mark
keywords_value = [kw.replace("…", "...") for kw in keywords_value] # \u2026 horizontal ellipsis

# remove mathematics
keywords_value = [kw.replace("\\infty", "infinity") for kw in keywords_value]
keywords_value = [kw.replace("\\mathrm{", "") for kw in keywords_value]
keywords_value = [kw.replace("{", "") for kw in keywords_value]
keywords_value = [kw.replace("}", "") for kw in keywords_value]

# remove '"'
keywords_value = [kw.replace('"', "") for kw in keywords_value]

# reduce needless blanks.
keywords_value = [re.sub("\s+", " ", kw) for kw in keywords_value]
keywords_value = [kw.replace(" -", "-") for kw in keywords_value]
keywords_value = [kw.replace("- ", "-") for kw in keywords_value]
keywords_value = [kw.replace(" //", "//") for kw in keywords_value]
keywords_value = [kw.replace("// ", "//") for kw in keywords_value]

# create single-word keyword
keywords_single = []
for kw in keywords_value:
    if len(kw.split()) == 1 and len(kw) <= 4:
        keywords_single.append(kw)

# Convert plural to singular
Lem = WordNetLemmatizer()
keywords_plural = {}
keywords_value_ = []
for i, kw in enumerate(keywords_value):
    if (kw not in keywords_single) and (len(kw.split(' ')) > 1):
        kw_ = ' '.join(kw.split(' ')[:-1] + [Lem.lemmatize(kw.split(' ')[-1])])
        keywords_value_.append(kw_)
    else:
        keywords_value_.append(kw)
        
    kw_ = kw.split(' ')
    for kw__ in kw_:
        if kw__ not in keywords_single and Lem.lemmatize(kw__) != kw__:
            keywords_plural.update({kw__: Lem.lemmatize(kw__)})

keywords_value = deepcopy(keywords_value_)
        
    
# keywords_value = [' '.join(kw.split(' ')[:-1] + [Lem.lemmatize(kw.split(' ')[-1])] if len(kw.split(' ')) > 1 and not in keywords_single else kw for kw in keywords_value]

keywords_value = [kw.lstrip(' ').rstrip(' ') for kw in keywords_value]
    
# create dash dictionary
for kw in keywords_value:
    kw_ = kw.split(" ")
    for kw__ in kw_:
        if ('-' in kw__) or ('//' in kw__):
            keywords_dash.append(kw__)
    
keywords_dash = np.unique(keywords_dash)

# create dictionary
keywords_dict = dict(zip(keywords_raw, keywords_value))
with open(keyword_dict_name, "w") as j:
    json.dump(keywords_dict, j, ensure_ascii=False, indent=2)
    
# create plural-singular dictionary
plural_sorted = dict(sorted(keywords_plural.items()))
with open(keyword_plural_name, "w") as j:
    json.dump(plural_sorted, j, ensure_ascii=False, indent=2)

# create abbreviation dictionary
with open(keyword_abb_name, "w") as j:
    json.dump(keywords_abb, j, ensure_ascii=False, indent=2)

# create "dash" keywords list
with open(keyword_dash_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords_dash:
        f.write(f"{keyword}\n")

# create "dash" keywords list
with open(keyword_singe_name, "w", encoding="utf-8-sig") as f:
    for keyword in keywords_single:
        f.write(f"{keyword}\n")

kc= (psa)
kc= (fpsa)
kc= (co2)
kc= (dc)
kc= (pv)
kc= (gis)
kc= (dcmi)
kc= (a)
kc= (vr)
kc= (aa%)
kc= (abc)
kc= (abc-pso)
kc= (ac)
kc= (ac-dc)
kc= (ac/dc)
kc= (acs)
kc= (adaline)
kc= (adc)
kc= (adrc)
kc= (ae)
kc= (ago)
kc= (ai)
kc= (airs)
kc= (alo)
kc= (amb)
kc= (ami)
kc= (amps)
kc= (amps-1d)
kc= (anemos)
kc= (anfis)
kc= (anfis:)
kc= (ann)
kc= (ann,)
kc= (ann:)
kc= (ansys)
kc= (aod)
kc= (ap)
kc= (apf)
kc= (api)
kc= (apros)
kc= (ar)
kc= (ar:)
kc= (arduino)
kc= (arima)
kc= (arima:)
kc= (arimax)
kc= (arma)
kc= (arma:)
kc= (armax)
kc= (arx:)
kc= (as4777)
kc= (asic)
kc= (asme)
kc= (asp)
kc= (assert)
kc= (assert-pv)
kc= (atlantis)
kc= (atlas)
kc= (atp)
kc= (atp-emtp)
kc= (atpdraw)
kc= (ats)
kc= (auc)
kc= (aufls)
kc= (avr)
kc= (llcl)
kc= (dc-dc)
kc= (p&o)
kc= (caes)
kc= (mega2560)
kc= (uno)
kc= (mppt)
kc= (a-55)
kc= (a55)
kc= (plc)
kc= (bapv)
kc= (bbc)
kc= (bd+c)
kc= (bdf)
kc= (beccs)
kc= (bems)
kc= (bes)
kc= (bess)
kc= (beta)
kc= (bga)
kc= (bim)
kc= (bipv)
kc= (bipv/t)
kc= (bipvt)
kc= (bispv)

In [23]:
len("bsf".split(' '))

1

In [18]:
Lem.lemmatize("networks")

'artificial neural networks'

In [14]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

Lem = WordNetLemmatizer()

phrase = 'cobblers ants women boys needs finds binaries hobbies busses wolves'

words = phrase.split()
for word in words :
  lemword = Lem.lemmatize(word)
  print(lemword)

cobbler
ant
woman
boy
need
find
binary
hobby
bus
wolf


In [35]:
for key in keywords_raw[10:1000:100]:
    print(f"{key}: {keywords_dict[key]}")

1-d modeling: 1-d modeling
airs total column ozone: airs total column ozone
accuracy evaluation: accuracy evaluation
advanced alkaline electrolyzer: advanced alkaline electrolyzer
allocation: allocation
antarctic stratospheric circumpolar vortex: antarctic stratospheric circumpolar vortex
asynchrouns machine: asynchrouns machine
bipv performance: bipv performance
battery control algorithm: battery control algorithm
best-so-far abc: best-so-far abc


* edit `values` while keeping `keys`, 

In [17]:
# 1. words inside bracket removal
[re.sub(r'\([^)]*\)', "", keyword_).rstrip(' ').lstrip(' ').lower() for keyword_

In [32]:
keywords_raw[-10:]

['λ-ridge concentrators',
 'αβ0',
 'β-blocker',
 'ε-constraint and fuzzy decision making approaches',
 'ε-constraint technique',
 'μ',
 'μ-synthesis',
 'μ-analysis',
 'π line model',
 '\ufeffforecasting']